# Experiments with custom ollama wrapper

In [ ]:
import ollama
import os
# Force bypass of proxy
os.environ["no_proxy"] = "192.168.13.13"
# Define multiple Ollama clients for different machines
global_search_client = ollama.Client(host='http://192.168.13.13:11434')  # Machine 1 (GraphRAG Global Search)
# local_search_client = ollama.Client(host='http://192.168.1.101:11434')  # Machine 2 (GraphRAG Local Search)

# Use them accordingly
response_global = global_search_client.chat(model='phi4:latest', messages=[{'role': 'user', 'content': 'What is knowledge graph?'}])
# response_local = local_search_client.chat(model='graphrag-local', messages=[{'role': 'user', 'content': 'Find similar entities to "GraphRAG".'}])

print("Global Search Response:", response_global['message']['content'])
# print("Local Search Response:", response_local['message']['content'])


Global Search Response: A knowledge graph is a structured representation of information that models real-world entities and their interrelationships. It consists of nodes (entities) and edges (relationships) to create a network that encodes data in a way that allows for semantic queries and reasoning.

Key characteristics of knowledge graphs include:

1. **Entities**: These are the basic units or objects, such as people, places, things, or concepts.
2. **Relationships**: These define how entities are connected, indicating various types of interactions or associations between them.
3. **Attributes/Properties**: Additional information about entities and relationships can be stored to enrich the graph.

Knowledge graphs enable sophisticated data integration, retrieval, and analysis by allowing machines to understand context, semantics, and concepts in a manner similar to human understanding. They are often powered by technologies like RDF (Resource Description Framework), SPARQL (a query 

now we understood how to configure ollama instances with custom client ip. Next we need to integrate this into a class.

In [ ]:
from langchain_ollama import OllamaLLM
import ollama
from pydantic import Field, ConfigDict
import os

class OllamaClient:
    def __init__(self, instances):
        """
        Initializes multiple Ollama clients for different machines.

        Args:
        - instances (dict): A dictionary mapping models to their respective Ollama server URLs.
                            Example: {"graphrag-global": "http://192.168.13.13:11434"}
        """
        self.clients = {model: ollama.Client(host=host) for model, host in instances.items()}
        
        # Ensure proxy is bypassed for local network
        os.environ["NO_PROXY"] = ",".join(instances.values())

    def __call__(self, prompt,model="phi4:latest",):
        """
        Sends a chat request to the appropriate Ollama instance based on the model.

        Args:
        - model (str): The name of the model (e.g., 'graphrag-global').
        - prompt (str): The user query.

        Returns:
        - str: The model's response.
        """
        if model not in self.clients:
            raise ValueError(f"No Ollama instance configured for model '{model}'")

        response = self.clients[model].chat(
            model=model, messages=[{"role": "user", "content": prompt}]
        )
        return response["message"]["content"]
    
    def list_models(self, model):
        """
        Lists available models from a specific Ollama instance.

        Args:
        - model (str): The model associated with a specific Ollama instance.

        Returns:
        - list: A list of available models.
        """
        if model not in self.clients:
            raise ValueError(f"No Ollama instance configured for model '{model}'")

        return self.clients[model].list()


# Force bypass of proxy
os.environ["no_proxy"] = "192.168.13.13"
os.environ["HTTP_PROXY"] = ""
os.environ["HTTPS_PROXY"] = ""
# Define Ollama instances
global_search_llm = OllamaClient({"phi4:latest": "http://192.168.13.13:11434"})

# Test the instances
response_global = global_search_llm(prompt="What is a knowledge graph?")


print("Global Search Response:", response_global)


Global Search Response: A knowledge graph is a structured representation of information that illustrates the relationships between entities (such as people, places, things) and the concepts they represent. It organizes data in a way that makes it easier to understand how different pieces of information are interconnected. Here's a breakdown of its key components:

1. **Entities**: These are the primary objects or subjects within the graph. Each entity is typically represented by a node.

2. **Attributes**: Entities can have attributes or properties that provide additional details about them, similar to fields in a database.

3. **Relationships/Edges**: These are connections between entities that describe how they relate to each other. They help illustrate the context and interactions between nodes.

4. **Schema/Ontology**: This defines the types of entities, relationships, and attributes included in the knowledge graph, providing structure and meaning.

5. **Data Sources**: Knowledge g

Now we need to make the class a langchain runnable

In [ ]:
import ollama
import os
from langchain.schema.runnable import RunnableSerializable
from pydantic import Field
from langchain.schema.runnable import RunnableLambda

import ollama
import os
from langchain.schema.runnable import RunnableSerializable
from pydantic import Field

class MultiOllamaClient(RunnableSerializable):
    clients: dict = Field(default_factory=dict, exclude=True)  # Exclude from Pydantic
    instances: dict = Field(default_factory=dict, exclude=True)  # Exclude from Pydantic
    temperature: float = Field(default=0.2, exclude=True)  # Exclude from Pydantic

    def __init__(self, **kwargs):
        """
        Initializes multiple Ollama clients for different machines.

        Args:
        - instances (dict): A dictionary mapping models to their respective Ollama server URLs.
                            Example: {"graphrag-global": "http://192.168.13.13:11434"}
        - temperature (float): The temperature setting for the LLM responses.
        """
        instances = kwargs.get("instances", None)
        if instances is None:
            raise ValueError("You must provide an 'instances' dictionary.")

        object.__setattr__(self, "clients", {model: ollama.Client(host=host) for model, host in instances.items()})
        object.__setattr__(self, "temperature", kwargs.get("temperature", 0.2))  # ✅ Fixed Assignment

    def __call__(self, input_data):
        """
        Handles LangChain-style input.

        Args:
        - input_data (dict): Expected format {"model": "model_name", "prompt": "query"}

        Returns:
        - str: The model's response.
        """
        if isinstance(input_data, str):
            raise ValueError("Expected dictionary input with 'model' and 'prompt' keys.")

        model = input_data.get("model", "phi4:latest")
        prompt = input_data.get("prompt")

        if model not in self.clients:
            raise ValueError(f"No Ollama instance configured for model '{model}'")

        response = self.clients[model].chat(
            model=model, messages=[{"role": "user", "content": prompt}], options={"temperature": self.temperature}
        )
        return response["message"]["content"]
    
    def invoke(self, input_data):
        """
        Handles LangChain-style input.

        Args:
        - input_data (dict): Expected format {"model": "model_name", "prompt": "query"}

        Returns:
        - str: The model's response.
        """
        return self.__call__(input_data)  # ✅ Now calls __call__()

    def list_models(self, model):
        """
        Lists available models from a specific Ollama instance.

        Args:
        - model (str): The model associated with a specific Ollama instance.

        Returns:
        - list: A list of available models.
        """
        if model not in self.clients:
            raise ValueError(f"No Ollama instance configured for model '{model}'")

        return self.clients[model].list()



# Force bypass of proxy
# os.environ["no_proxy"] = "192.168.13.13"
# Instead just set following through terminal
# conda config --set proxy_servers.http "" 
# conda config --set proxy_servers.https ""

# Define Ollama instances dynamically
ollama_runnable = MultiOllamaClient(instances={"phi4:latest": "http://192.168.13.13:11434"},temperature=0.9)

# Use inside a Runnable Chain
chain = RunnableLambda(ollama_runnable)

# Invoke the chain
response_global = chain.invoke({"prompt": "What is a knowledge graph?"})


print("Global Search Response:", response_global)



Global Search Response: A knowledge graph is a data structure that organizes and represents information in a way that highlights the relationships between different entities. It consists of nodes (entities) and edges (relationships or connections) to form a network that can be used for various applications, such as enhancing search engine capabilities, improving recommendation systems, or powering artificial intelligence models.

Key features of knowledge graphs include:

1. **Entities**: These are objects, concepts, people, places, or any other significant items in the data domain being represented. They serve as nodes within the graph.

2. **Relationships**: Edges connect entities and denote the relationships between them. These can describe various types of associations like "is a part of," "created by," "located in," etc.

3. **Attributes/Properties**: Entities often have associated properties or attributes, such as a person's birthdate or a location’s coordinates.

4. **Interopera